In [1]:
from dotenv import load_dotenv
import os

# Cargar las variables del archivo .env
load_dotenv()

# Obtener la clave de OpenAI
openai_api_key = os.getenv("OPENAI_API_KEY")

# Verifica que la clave se ha cargado correctamente
if openai_api_key:
    print("La clave de OpenAI se ha cargado correctamente.")
else:
    print("Error al cargar la clave de OpenAI.")

La clave de OpenAI se ha cargado correctamente.


In [2]:
from openai import OpenAI
import os
client=OpenAI(api_key=openai_api_key)
MODEL='gpt-4-turbo'

In [3]:
import yt_dlp

# URL del vídeo de YouTube
url = "https://www.youtube.com/watch?v=G9FRadIkkE0"

# Usar yt-dlp para obtener la información del vídeo
ydl_opts = {}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(url, download=False)
    video_title = info_dict.get('title', None)
    video_views = info_dict.get('view_count', None)
    video_length = info_dict.get('duration', None)
    video_thumbnail = info_dict.get('thumbnail', None)

# Imprimir la información obtenida
print(f"Title: {video_title}")
print(f"Views: {video_views}")
print(f"Length: {video_length} seconds")
print(f"Thumbnail: {video_thumbnail}")


[youtube] Extracting URL: https://www.youtube.com/watch?v=G9FRadIkkE0
[youtube] G9FRadIkkE0: Downloading webpage
[youtube] G9FRadIkkE0: Downloading ios player API JSON
[youtube] G9FRadIkkE0: Downloading mweb player API JSON
[youtube] G9FRadIkkE0: Downloading m3u8 information
Title: Microsoft Recall is MANDATORY
Views: 234578
Length: 299 seconds
Thumbnail: https://i.ytimg.com/vi/G9FRadIkkE0/maxresdefault.jpg


In [4]:
#descargar vídeo con yt-dlp

# Opciones para descargar el vídeo
ydl_opts = {
    'format': 'best',  # Seleccionar la mejor calidad disponible
    'outtmpl': '%(title)s.%(ext)s',  # Guardar el archivo con el título del vídeo
}

# Descargar el vídeo
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])


[youtube] Extracting URL: https://www.youtube.com/watch?v=G9FRadIkkE0
[youtube] G9FRadIkkE0: Downloading webpage
[youtube] G9FRadIkkE0: Downloading ios player API JSON
[youtube] G9FRadIkkE0: Downloading mweb player API JSON
[youtube] G9FRadIkkE0: Downloading m3u8 information
[info] G9FRadIkkE0: Downloading 1 format(s): 18
[download] Destination: Microsoft Recall is MANDATORY.mp4
[download] 100% of   10.79MiB in 00:00:00 at 37.60MiB/s    


In [3]:
import yt_dlp
import re

def descargar_solo_audio(url):
    # Función para limpiar el título del vídeo
    def sanitize_filename(filename):
        # Reemplaza los espacios por guiones bajos
        filename = filename.replace(" ", "_")
        # Elimina caracteres no alfanuméricos, dejando solo letras, números y guiones bajos
        filename = re.sub(r'[^\w-]', '', filename)
        return filename

    print("descargar_solo_audio...")

    # Obtener la información del vídeo y el título
    ydl_opts_info = {}
    with yt_dlp.YoutubeDL(ydl_opts_info) as ydl:
        info_dict = ydl.extract_info(url, download=False)
        video_title = info_dict.get('title', None)

    # Limpiar el título del vídeo
    clean_title = sanitize_filename(video_title)

    # Opciones para descargar solo el audio
    ydl_opts = {
        'format': 'bestaudio/best',  # Descargar la mejor calidad de audio disponible
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',  # Puedes cambiar a 'wav' o 'm4a' si prefieres otro formato
            'preferredquality': '192',  # Calidad de audio
        }],
        'outtmpl': f'{clean_title}.%(ext)s',  # Guardar el archivo con el título limpio
        'quiet': True,  # Esta opción desactiva los mensajes de yt-dlp
        'no_warnings': True  # Desactiva las advertencias
    }

    # Descargar el audio
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

# Devolver el nombre del archivo
    filename = f"{clean_title}.mp3"
    print(f"Descarga de audio completa. Archivo guardado como: {filename}")
    return filename
# Ejemplo de uso:
# descargar_solo_audio("https://www.youtube.com/watch?v=G9FRadIkkE0")


In [ ]:
descargar_solo_audio("https://www.youtube.com/watch?v=wRsX_ZkzxvQ")

descargar_solo_audio...


[youtube] Extracting URL: https://www.youtube.com/watch?v=wRsX_ZkzxvQ
[youtube] wRsX_ZkzxvQ: Downloading webpage
[youtube] wRsX_ZkzxvQ: Downloading ios player API JSON
[youtube] wRsX_ZkzxvQ: Downloading mweb player API JSON
[youtube] wRsX_ZkzxvQ: Downloading m3u8 information


In [21]:

from pydub import AudioSegment
import os

import os
from pydub import AudioSegment

def sanitize_filename(filename):
    # Reemplaza los espacios por guiones bajos
    filename = filename.replace(" ", "_")
    # Elimina caracteres no alfanuméricos, dejando solo letras, números y guiones bajos
    filename = re.sub(r'[^\w-]', '', filename)
    return filename

# Función para dividir el audio en segmentos de 120 segundos por defecto
def split_audio(file_path, output_dir="output", base_name=None, chunk_length=120):
     # Si no se proporciona base_name, obtenerlo del file_path
    if base_name is None:
        base_name = os.path.splitext(os.path.basename(file_path))[0]  # Extraer nombre de archivo sin extensión
    
    # Cargar el archivo de audio
    audio = AudioSegment.from_mp3(file_path)
    duration_ms = len(audio)  # Duración total del audio en milisegundos
    chunk_length_ms = chunk_length * 1000  # Convertir la longitud de los fragmentos a milisegundos

    # Crear el directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    chunk_files = []

    # Dividir el audio en fragmentos estándar de chunk_length_ms
    for i, start in enumerate(range(0, duration_ms, chunk_length_ms)):
        print(f"\rFragmento {i+1}/{int(duration_ms/chunk_length_ms)}", end='')
        chunk = audio[start:start + chunk_length_ms]
        chunk_name = f"{i:03d}_{base_name}.mp3"
        chunk_path = os.path.join(output_dir, chunk_name)
        chunk.export(chunk_path, format="mp3")
        chunk_files.append(chunk_path)
    
    return chunk_files

# Ejemplo de uso:
# Dividir un archivo de audio en segmentos de 120 segundos y guardarlos en una carpeta
# split_audio("audio.mp3", "output_dir", "segment", chunk_length=120)


def transcribe(audio_file,output_dir="transcribed_output"):
    if not os.path.isfile(audio_file):
        print(f"El archivo {audio_file} no existe.")
        return  False
    
    # Crear el directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)
    
    with open(audio_file, 'rb') as f:
        print("Transcribiendo audio...",end='')
        transcript = client.audio.transcriptions.create(
            model='whisper-1',
            file=f
        )
        print(f"Transcripción completa {transcript.text}")
        name,extension = os.path.splitext(audio_file)
        transcribed_file = f"transcribed_{name}.txt"
        print(f"Archivo de transcripción guardado como: {transcribed_file}")
        with open(transcribed_file, 'w') as transcription_file:
            transcription_file.write(transcript.text)
    return transcribed_file

def resumen_transcripcion(file,idioma="Castellano"):
    #enviar la transcripción a openAI para obtener el resumen
    MODEL='gpt-4o'
    system_role=f'''
    Eres un asistente para resumir. Te pasan la transcripción de un vídeo y respondes
    con un texto donde explicas los contenidos del video.
    Empiezas con una introducción, siguies con los apartados principales y terminas con una conclusión.
    Devuelves el texto en formato markdown y solamente escribes el contenido del texto.
    Siempre respondes en {idioma}
    
    
    '''

    # Leer el contenido de cada archivo y agregarlo a la transcripción total
    texto_a_resumir=""
    with open(file, 'r') as f:
        content = f.read()
        texto_a_resumir += content + "\n"
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {'role':'system','content':system_role},
            {'role':'user','content':texto_a_resumir},
        ],
    )
    print(response.choices[0].message.content)
    file_resumed="resumed_.txt"
    with open(file_resumed, 'w') as transcription_file:
        transcription_file.write(response.choices[0].message.content)
    return response.choices[0].message.content


def guion_video(file,idioma="Castellano"):
    #enviar la transcripción a openAI para obtener el resumen
    MODEL='gpt-4o'
    system_role=f'''
    Eres un guionista para vídeos de youtube. Te pasan la transcripción de un vídeo y respondes
    con un guión del vídeo en el idioma {idioma} con frases elaboradas.
    Tienes que presentar un guión con los minutos más exactos para que 
    se pueda grabar un nuevo vídeo con el mismo contenido pero en el idioma {idioma}.
    Devuelves el texto en formato markdown.
    
    '''

    # Leer el contenido de cada archivo y agregarlo a la transcripción total
    texto_a_resumir=""
    with open(file, 'r') as f:
        content = f.read()
        texto_a_resumir += content + "\n"
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {'role':'system','content':system_role},
            {'role':'user','content':texto_a_resumir},
        ],
    )
    print(response.choices[0].message.content)
    file_resumed=f"guion_{file}"
    with open(file_resumed, 'w') as transcription_file:
        transcription_file.write(response.choices[0].message.content)
    return response.choices[0].message.content



import os

# Unificar el contenido de todos los archivos en uno solo
def unificar_archivos(files, output_file="transcripcion_unificada.txt"):
    total_transcription = ""
    
    file_name = files[0]  # Obtener el nombre del archivo dentro de la lista
    directori=file_name.split('/')[0]
    base_name=file_name.split('/')[1][4::].split('.')[0]
    output_file=f"{directori}/transcribed_{base_name}.txt"
    for file in files:
  
        
        
        # Leer el contenido de cada archivo y agregarlo a la transcripción total
        with open(file, 'r') as f:
            content = f.read()
            total_transcription += content + "\n"
    
    # Guardar la transcripción unificada en un archivo
    with open(output_file, 'w') as out_file:
        out_file.write(total_transcription)
    
    print(f"Archivos unificados y guardados en: {output_file}")


def borrar_archivos(files):
    for file in files:
        try:
            os.remove(file)  # Eliminar el archivo
            print(f"Archivo eliminado: {file}")
        except FileNotFoundError:
            print(f"El archivo no existe: {file}")
        except Exception as e:
            print(f"No se pudo eliminar el archivo {file} debido a: {e}")






        





In [8]:
segments=split_audio("Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.mp3")
# segments=split_audio("Dr_Craig_Koniver_Peptide__Hormone_Therapies_for_Health_Performance__Longevity.mp3")

Fragmento 5/4

In [39]:
segments

['output/000_Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.mp3',
 'output/001_Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.mp3',
 'output/002_Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.mp3',
 'output/003_Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.mp3',
 'output/004_Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.mp3']

In [14]:
trasc_files=[]
for segment in segments:
    print(f"Transcribiendo segmento: {segment}")
    file_transcribed=transcribe(segment)
    trasc_files.append(file_transcribed)
print(trasc_files)


    

Transcribiendo segmento: output/000_Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.mp3
Transcribiendo audio...Transcripción completa The major effect of testosterone is to make effort feel good. That's what testosterone does. The real question is, are you enjoying what you're doing? When effort feels good, life just gets way better. So there's something about these molecules that in an ancient way are linked to the will to live. They're linked to effort and making effort feel good. That's the Holy Grail. There's a very interesting relationship between testosterone and dopamine. So dopamine and testosterone are closely related in the pituitary system. And obviously testosterone comes from the adrenals and from the testes. But the major effect of testosterone is to make effort feel good. That's what testosterone does. It has other effects too, right? Reproductive effects, androgenizing parts of the body, et cetera, but it makes effort feel good. The testosterone molecule

In [36]:
unificar_archivos(trasc_files)


Archivos unificados y guardados en: transcribed_output/transcribed_Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.txt


In [32]:
file_name='output/000_Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.mp3'
directori=file_name.split('/')[0]
base_name=file_name.split('/')[1][4::].split('.')[0]
output_file=f"{directori}/transcribed_{base_name}.txt"
print(output_file)

output/transcribed_Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.txt


In [40]:
borrar_archivos(segments)

Archivo eliminado: output/000_Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.mp3
Archivo eliminado: output/001_Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.mp3
Archivo eliminado: output/002_Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.mp3
Archivo eliminado: output/003_Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.mp3
Archivo eliminado: output/004_Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.mp3


In [6]:
#prueba todo
new_file_name=descargar_solo_audio("https://www.youtube.com/watch?v=G9FRadIkkE0")

descargar_solo_audio...
[youtube] Extracting URL: https://www.youtube.com/watch?v=G9FRadIkkE0
[youtube] G9FRadIkkE0: Downloading webpage
[youtube] G9FRadIkkE0: Downloading ios player API JSON
[youtube] G9FRadIkkE0: Downloading mweb player API JSON
[youtube] G9FRadIkkE0: Downloading m3u8 information
Descarga de audio completa. Archivo guardado como: Microsoft_Recall_is_MANDATORY.mp3


In [7]:
new_file_name

'Microsoft_Recall_is_MANDATORY.mp3'

In [8]:
transcribed_file=transcribe(new_file_name)

Transcribiendo audio...Transcripción completa This is a video I really didn't think I was gonna make. I'm on my inside computer right now and I ran into this issue and it has direct implications with Microsoft Recall. Specifically how Microsoft assured us that it wouldn't infiltrate regular systems that was only for like AI arm based systems with the Recall or Copilot Plus chips. I can say definitively today that was incorrect. Recall is getting installed on every single system in 24h2 more so and this is really really important. It is a dependency in File Explorer that should alarm a lot of people and I wanted to walk you through all the things that you need to do to disable it but also the future of Windows and how we think about Windows because it's really important that one if you continue using Windows in 24h2 and beyond you disable the Recall service but I just want to show you the findings and then you can come to whatever conclusion you want to come to as I'm just more concerne

In [13]:
transcribed_file

'transcribed_Microsoft_Recall_is_MANDATORY.txt'

In [16]:
resumen_transcripcion(transcribed_file)

En el vídeo se discute una preocupación importante sobre un cambio en el software de Windows que podría afectar a muchos usuarios. A lo largo del video, se destacan implicaciones directas relacionadas con el servicio "Recall" de Microsoft.

**Introducción:** El orador comienza mencionando su sorpresa por la necesidad de hacer este video, debido a un problema encontrado con la integración del servicio "Recall" de Microsoft en sistemas tradicionales, a pesar de las garantías iniciales de que solo afectaría a sistemas específicos con chips de inteligencia artificial (IA) armados. Este problema se vuelve relevante para la próxima actualización Windows 24h2.

**Apartados principales:**

1. **Implicaciones de Recall en Windows:** Se explica cómo el servicio "Recall" se está instalando en todos los sistemas, no solo en aquellos con soporte de inteligencia artificial, lo que podría ser alarmante. "Recall" se convierte en una dependencia dentro del Explorador de Archivos de Windows, lo que moti

'En el vídeo se discute una preocupación importante sobre un cambio en el software de Windows que podría afectar a muchos usuarios. A lo largo del video, se destacan implicaciones directas relacionadas con el servicio "Recall" de Microsoft.\n\n**Introducción:** El orador comienza mencionando su sorpresa por la necesidad de hacer este video, debido a un problema encontrado con la integración del servicio "Recall" de Microsoft en sistemas tradicionales, a pesar de las garantías iniciales de que solo afectaría a sistemas específicos con chips de inteligencia artificial (IA) armados. Este problema se vuelve relevante para la próxima actualización Windows 24h2.\n\n**Apartados principales:**\n\n1. **Implicaciones de Recall en Windows:** Se explica cómo el servicio "Recall" se está instalando en todos los sistemas, no solo en aquellos con soporte de inteligencia artificial, lo que podría ser alarmante. "Recall" se convierte en una dependencia dentro del Explorador de Archivos de Windows, lo q

In [18]:
new_file_name=descargar_solo_audio("https://www.youtube.com/watch?v=402y2mVDbdo")
transcribed_file=transcribe(new_file_name)
resumen_transcripcion(transcribed_file,idioma="Català")

descargar_solo_audio...
[youtube] Extracting URL: https://www.youtube.com/watch?v=402y2mVDbdo
[youtube] 402y2mVDbdo: Downloading webpage


[youtube] 402y2mVDbdo: Downloading ios player API JSON
[youtube] 402y2mVDbdo: Downloading mweb player API JSON
[youtube] 402y2mVDbdo: Downloading m3u8 information
Descarga de audio completa. Archivo guardado como: Windows_Recall_Mandatory_Time_to_Switch_to_Linux.mp3
Transcribiendo audio...Transcripción completa Today, I wanna make the case to switch to Linux from Windows as we discussed in the past, retrace your steps with Recall. This is a feature that's getting introduced to Windows and the Copilot PCs that allows you to search across time and find the content that you need. While this sounds fascinating and great to automatically just pull up something that you've done in the past, with this cool new Recall feature, we've spoke about many different problems with saving screenshots and snapshots of your particular screen in this Recall feature so that it can actually do this type of Recall and tell you where and what time you were performing some sort of an action. And today, I wanna

'En aquest vídeo es discuteix sobre la controvèrsia al voltant de la nova funció "Recall" de Windows i es fa un argument per canviar a Linux. \n\n### Introducció\nL\'autor comença ressaltant una nova funció de Windows anomenada "Recall", que permet al sistema operatiu fer captures de pantalla per ajudar l\'usuari a recordar i buscar informació de manera retroactiva. Tot i que a primera vista pot semblar una característica útil, l\'autor exposa les seves preocupacions sobre els problemes de privadesa i seguretat associats amb aquesta funció.\n\n### Funció "Recall" i Problemes de Seguretat\nLa funció "Recall" crea captures de pantalla de manera automàtica per proporcionar un context que permet trobar informació relacionada amb les activitats passades de l\'usuari. No obstant això, es menciona que aquesta característica podria ser una porta oberta als hackers, ja que la informació capturada es desa en una base de dades SQLite, la qual pot ser vulnerable a atacs.\n\n### Controvèrsia pel De

In [22]:
guion_video(transcribed_file)

```markdown
# 🎥 ¿Por Qué Deberías Cambiar de Windows a Linux? 📉

## Introducción

**[00:00]** ¡Hola a todos! Hoy quiero presentarles por qué deberíamos considerar cambiar de Windows a Linux, principalmente debido a las últimas controversias que han surgido con la nueva característica de Windows, el "Recall". Acompáñenme mientras desglosamos este tema y evaluamos si vale la pena el cambio.

## La Función de Recall en Windows

**[00:20]** Como muchos ya saben, Windows está introduciendo una nueva función, el "Recall", que permite buscar contenido que has visto en el pasado y recuperar instantáneas de tu pantalla. Aunque suena fascinante, esta función también acarrea serias preocupaciones sobre privacidad y seguridad.

**[01:00]** Hemos discutido en el pasado cómo "Recall" guarda capturas de pantalla continuamente, lo que podría ser explotado por hackers, especialmente después de que se descubriera una vulnerabilidad en la base de datos SQLite donde Recall almacena su información.

## Pro

'```markdown\n# 🎥 ¿Por Qué Deberías Cambiar de Windows a Linux? 📉\n\n## Introducción\n\n**[00:00]** ¡Hola a todos! Hoy quiero presentarles por qué deberíamos considerar cambiar de Windows a Linux, principalmente debido a las últimas controversias que han surgido con la nueva característica de Windows, el "Recall". Acompáñenme mientras desglosamos este tema y evaluamos si vale la pena el cambio.\n\n## La Función de Recall en Windows\n\n**[00:20]** Como muchos ya saben, Windows está introduciendo una nueva función, el "Recall", que permite buscar contenido que has visto en el pasado y recuperar instantáneas de tu pantalla. Aunque suena fascinante, esta función también acarrea serias preocupaciones sobre privacidad y seguridad.\n\n**[01:00]** Hemos discutido en el pasado cómo "Recall" guarda capturas de pantalla continuamente, lo que podría ser explotado por hackers, especialmente después de que se descubriera una vulnerabilidad en la base de datos SQLite donde Recall almacena su informac

In [25]:
url="https://youtu.be/q3J1ADFfb7c?si=gb1buPQn8ktQPBsO"
new_file_name=descargar_solo_audio(url)
transcribed_file=transcribe(new_file_name)
resumen_transcripcion(transcribed_file)


descargar_solo_audio...
[youtube] Extracting URL: https://youtu.be/q3J1ADFfb7c?si=gb1buPQn8ktQPBsO
[youtube] q3J1ADFfb7c: Downloading webpage


[youtube] q3J1ADFfb7c: Downloading ios player API JSON
[youtube] q3J1ADFfb7c: Downloading mweb player API JSON
[youtube] q3J1ADFfb7c: Downloading m3u8 information
Descarga de audio completa. Archivo guardado como: SER_vs_ESTAR_en_español_Cuándo_se_usan_Cómo_se_usan_Cuáles_son_las_diferencias_Explicación_.mp3
Transcribiendo audio...Transcripción completa Hola a todos, bienvenidos otra vez a la Burbuja del Español. Yo soy Marta, soy vuestra profesora de español y en el vídeo de hoy vamos a estudiar una cosa muy importante, la diferencia entre los verbos SER y ESTAR. En otros vídeos anteriores hemos estudiado el verbo SER y el verbo ESTAR. Ahora podéis ir a ver el vídeo para repasar un poco. Hoy vamos a estudiar sólo la diferencia, porque se utilizan SER en unos casos y ESTAR en otros. Vamos a ver en cuáles. El verbo SER se utiliza para las características. El verbo SER define las características de las personas o de las cosas. Por ejemplo, yo soy una chica, yo soy española, esto es una p

'En el vídeo presentado por Marta, de La Burbuja del Español, se explora uno de los temas más desafiantes para los estudiantes de español: la diferencia entre los verbos "ser" y "estar". Ambos verbos significan "to be" en inglés, pero su uso varía según el contexto y el significado deseado.\n\n### Apartados Principales:\n\n1. **Diferencias Básicas:**\n   - **Ser:** Se utiliza para describir características permanentes de personas o cosas. Por ejemplo, "yo soy española" o "esto es una pizarra".\n   - **Estar:** Indica estados temporales de personas o cosas, tal como "mi amiga está contenta" o "la pizarra está sucia".\n\n2. **Uso en Profesiones:**\n   - **Ser:** Se emplea para hablar de profesiones habituales. Ejemplo: "yo soy profesora".\n   - **Estar:** Describe ocupaciones temporales, como en "estoy de camarera".\n\n3. **Localización de Eventos y Personas:**\n   - **Ser:** Se utiliza para situar un evento, como "la reunión es en la facultad de idiomas".\n   - **Estar:** Se usa para ub